# This is a notebook for Melanie to study how nominal MPCs work. This is an example following [K. Chan's MPC tutorial](https://github.com/kchan45/MPC_tutorials/blob/main/Python/mpc_demo.ipynb). 

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from casadi import *

### Aim: Understand how to set up MPC. Goal of MPC is to solve finite-time OCP at each time step, take optimal inputs obtained from solution of this OCP and use them to control the system. Then, feedback from the system will be incorporated int othe next OCP.

System information: 
- nominal MPC (e.g. no noise or disturbances in process.)
- OCP: double integrator model
    - discrete-time system, linear model
    - wish to solve minimization s.t. u,x in the form:
        - $\min_{x, u} x_{N}^{\top}Q_{N}x_{N} + \sum_{k=0}^{N-1} x_{k}^{\top}Qx_{k}+u_{k}^{\top}Ru_{k}$
        - s.t.:
            - $|u_{k}| \leq 1 \quad \quad \forall k = 0, 1,...,N-1$
            - $|y_{k}| \leq 1 \quad \quad \forall k = 0, 1,...,N-1$
        - $x(i+1) = A x(i) + B u(i)$
        - $y(i) = C x(i)$
    - prediction horizon: $\quad 10$
    - terminal cost: $\qquad \quad x_{N}^{\top}Q_{N}x_{N}$
    - stage cost: $\qquad \qquad \sum_{k=0}^{N-1} x_{k}^{\top}Qx_{k}+u_{k}^{\top}Ru_{k}$
- using off-the-shelf packages, CasADi functions to formulate the problem, Opti to solve

### Setting up the problem

Defining variables to formulate problem using helper CasADI functions

In [39]:
x_states = 2 # total number of states
x = SX.sym('x', x_states) # symbol for states

u_inputs = 1 # total number of inputs
u = SX.sym('u', u_inputs) # symbol for inputs

y_outputs = 1 # total number of outputs

Defining the arrays for the double integrator problem 

In [40]:
A, B, C = array([[1, 1], [0,1]]), array([[0.5], [1.0]]), array([[1, 0]])

Defining evolution equations 

In [41]:
f = Function('f', [x, u], [A @ x + B @ u]) # dynamics, e.g. state evolution
g = Function('g', [x], [C @ x ]) # output evolution 

Defining cost functions

In [42]:
Q, QN, R = np.eye(x_states), np.eye(x_states), np.eye(u_inputs) # weights for the cost
    # note that np.eye creates a 2D array with 1s on the diagonal, 0s else {https://numpy.org/devdocs/reference/generated/numpy.eye.html}
stage_cost = Function('stage_cost', [x, u], [x.T @ R @ u])
terminal_cost = Function('terminal_cost', [x], [x.T @ QN @ x])

Defining constraints and initial values of the decision variables

In [44]:
U_bound, Y_bound = 1., 1. # Check with K.C. --> Q: why are the bounds floats?; A: 
u_min, u_max = -U_bound * np.ones((u_inputs, 1)), U_bound * np.ones((u_inputs, 1))
y_min, y_max = -Y_bound * np.ones((y_outputs, 1)), Y_bound * np.ones((y_outputs, 1))

x_init, u_init, y_init = np.zeros(x_states), np.zeros(u_inputs), np.zeros(y_outputs) 
# NOTE: best to avoid making shallow/deep copy for learning purposes. Essentially all deep copies anw...

### Using Opti for OCP structure